## Thanks @abhishek for his kernel https://www.kaggle.com/abhishek/beating-the-benchmark-neural-network
## Due to 'wheezy-copper-turtle-magic' is categorical feature, I decided to add one-hot encoding. 

In [1]:
import os
import gc
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from keras.layers import Dense, Input
from collections import Counter
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import callbacks
from keras import backend as K
from keras.layers import Dropout

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


### Some functions to make life easier

In [2]:
def submit(predictions):
    submit = pd.read_csv('../input/sample_submission.csv')
    submit["target"] = predictions
    submit.to_csv("submission.csv", index=False)

def fallback_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except:
        return 0.5

def auc(y_true, y_pred):
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

### Load data

In [3]:
df_tr = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")

### Decalare variables

In [4]:
NFOLDS = 5
RANDOM_STATE = 42
numeric = [c for c in df_tr.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]

### One-hot encodings and basic statistic based on categorical column 'wheezy-copper-turtle-magic'

In [5]:
len_train = df_tr.shape[0]
df_test['target'] = -1
data = pd.concat([df_tr, df_test])
data['magic_count'] = data.groupby(['wheezy-copper-turtle-magic'])['id'].transform('count')
data = pd.concat([data, pd.get_dummies(data['wheezy-copper-turtle-magic'])], axis=1, sort=False)

df_tr = data[:len_train]
df_test = data[len_train:]

### Let's make KFold validation with 5 folds

In [6]:
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=RANDOM_STATE)

#### Clear garbage

In [7]:
gc.collect()

39

### Preparing data for Neural Network

In [8]:
y = df_tr.target
ids = df_tr.id.values
train = df_tr.drop(['id', 'target'], axis=1)
test_ids = df_test.id.values
test = df_test[train.columns]

oof_preds = np.zeros((len(train)))
test_preds = np.zeros((len(test)))

scl = preprocessing.StandardScaler()
scl.fit(pd.concat([train, test]))
train = scl.transform(train)
test = scl.transform(test)

### We all waited this moment! Training

In [9]:
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    print("Current Fold: {}".format(fold_))
    trn_x, trn_y = train[trn_, :], y.iloc[trn_]
    val_x, val_y = train[val_, :], y.iloc[val_]

    inp = Input(shape=(trn_x.shape[1],))
    x = Dense(2000, activation="relu")(inp)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(1000, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(500, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(100, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    out = Dense(1, activation="sigmoid")(x)
    clf = Model(inputs=inp, outputs=out)
    clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])

    es = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001, patience=10,
                                 verbose=1, mode='max', baseline=None, restore_best_weights=True)

    rlr = callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.5,
                                      patience=3, min_lr=1e-6, mode='max', verbose=1)

    clf.fit(trn_x, trn_y, validation_data=(val_x, val_y), callbacks=[es, rlr], epochs=100, batch_size=1024)
    
    val_preds = clf.predict(val_x)
    test_fold_preds = clf.predict(test)
    
    print("AUC = {}".format(metrics.roc_auc_score(val_y, val_preds)))
    oof_preds[val_] = val_preds.ravel()
    test_preds += test_fold_preds.ravel() / NFOLDS
    
    K.clear_session()
    gc.collect()

Current Fold: 0
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 209714 samples, validate on 52430 samples
Epoch 1/100
209714/209714 [==============================] - 7s 31us/step - loss: 0.7304 - auc: 0.5406 - val_loss: 0.6657 - val_auc: 0.6363
Epoch 2/100
209714/209714 [==============================] - 4s 19us/step - loss: 0.6611 - auc: 0.6419 - val_loss: 0.6310 - val_auc: 0.7006
Epoch 3/100
209714/209714 [==============================] - 4s 19us/step - loss: 0.6173 - auc: 0.7173 - val_loss: 0.6070 - val_auc: 0.7308
Epoch 4/100
209714/209714 [==============================] - 4s 19us/step - loss: 0.5700 - auc: 0.7738 - val_loss: 0.5892 - val_auc: 0.7578
Epoch 5/100
209714/209714 [==============================] - 4s 19us/step - loss: 0.5234 - auc: 0.8171 - val_loss: 0.5864 - val_auc: 0.767

### Make submition

In [10]:
submit(test_preds)

### If you found it usefull, upvote please :) 